# Deploy and score a machine learning model by using an online endpoint 

Learn how to use an online endpoint to deploy your model, so you don't have to create and manage the underlying infrastructure. You'll begin by deploying a model on your local machine to debug any errors, and then you'll deploy and test it in Azure.

Managed online endpoints help to deploy your ML models in a turnkey manner. Managed online endpoints work with powerful CPU and GPU machines in Azure in a scalable, fully managed way. Managed online endpoints take care of serving, scaling, securing, and monitoring your models, freeing you from the overhead of setting up and managing the underlying infrastructure. 

For more information, see [What are Azure Machine Learning endpoints?](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

## Prerequisites

* Set Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* To deploy locally, you must install Docker Engine on your local computer. We highly recommend this option, so it's easier to debug issues.

In [1]:
%pip install docker

Note: you may need to restart the kernel to use updated packages.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [6]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [3]:
# enter details of your AML workspace
subscription_id = "73fc0666-cbe3-4535-b3c6-83baeeaf94f5"
resource_group = "fhtw-solution-engineering"
workspace = "fhtw-ml-23"

In [7]:
# get a handle to the workspace
ml_client = MLClient(
    InteractiveBrowserCredential(), subscription_id, resource_group, workspace
)

## Deploy and debug locally by using local endpoints

### Note
* To deploy locally, [Docker Engine](https://docs.docker.com/engine/install/) must be installed.
* Docker Engine must be running. Docker Engine typically starts when the computer starts. If it doesn't, you can [troubleshoot Docker Engine](https://docs.docker.com/config/daemon/#start-the-daemon-manually).

# 2. Create local endpoint and deployment

## 2.1 Create local endpoint

The goal of a local endpoint deployment is to validate and debug your code and configuration before you deploy to Azure. Local deployment has the following limitations:
* Local endpoints *do not support* traffic rules, authentication, or probe settings.
* Local endpoints support only one deployment per endpoint

In [8]:
# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=local_endpoint_name, description="this is a sample local endpoint"
)

In [9]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (local-01090227674432) Done (0m 0s)
Field 'mirror_traffic': This is an experimental field, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'local-01090227674432', 'description': 'this is a sample local endpoint', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/home/codespace/.azureml/inferencing/local-01090227674432'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fae2f388c10>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## 2.2 Create local deployment

The example contains all the files needed to deploy a model on an online endpoint. To deploy a model, you must have:

* Model files (or the name and version of a model that's already registered in your workspace). In the example, we have a scikit-learn model that does regression.
* The code that's required to score the model. In this case, we have a score.py file.
* An environment in which your model runs. As you'll see, the environment might be a Docker image with Conda dependencies, or it might be a Dockerfile.
* Settings to specify the instance type and scaling capacity.

### Key aspects of deployment 

- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_configuration` - the configuration for the source code and scoring script
    - `path`- Path to the source code directory for scoring the model
    - `scoring_script` - Relative path to the scoring file in the source code directory
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

In [19]:
model = Model(path="regression_model.pkl")
env = Environment(
    conda_file="./conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(code="./", scoring_script="score.py"),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [20]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (local-01090227674432 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
 ---> 4bd2dec82540
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 05bf02425fc0
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> f95bdb4b71b1
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 928555e472d4
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in fae7563fc00d
.Collecting package metadata (repodata.json): ...working... ..Killed
The command '/bin/sh -c conda env create -n inf-conda-env --file conda.yml' returned a non-zero code: 137Done (0m 20s)


LocalEndpointImageBuildError: Building the local endpoint image failed with error: The command '/bin/sh -c conda env create -n inf-conda-env --file conda.yml' returned a non-zero code: 137

# 3. Verify the local deployment succeeded

## 3.1 Check the status to see whether the model was deployed without error

In [ ]:
ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

## 3.2 Get logs

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=local_endpoint_name, local=True, lines=50
)

## 3.3 Invoke the local endpoint
Invoke the endpoint to score the model by using the convenience command invoke and passing query parameters that are stored in a JSON file

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=local_endpoint_name,
    request_file="../model-1/sample-request.json",
    local=True,
)

# 4. Deploy your online endpoint to Azure
Next, deploy your online endpoint to Azure.

## 4.1 Configure online endpoint
`endpoint_name`: The name of the endpoint. It must be unique in the Azure region. Naming rules are defined under [managed online endpoint limits](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas#azure-machine-learning-managed-online-endpoints-preview).

`auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` does not expire, but `aml_token` does expire. 

Optionally, you can add description, tags to your endpoint.

In [22]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

## 4.2 Create the endpoint

Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [24]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ClientAuthenticationError: (InvalidAuthenticationTokenTenant) The access token is from the wrong issuer 'https://sts.windows.net/f8cdef31-a31e-4b4a-93e4-5f571e91255a/'. It must match the tenant 'https://sts.windows.net/a0a6d112-ab35-410a-b961-1280cf758bfc/' associated with this subscription. Please use the authority (URL) 'https://login.windows.net/a0a6d112-ab35-410a-b961-1280cf758bfc' to get the token. Note, if the subscription is transferred to another tenant there is no impact to the services, but information about new tenant could take time to propagate (up to an hour). If you just transferred your subscription and see this error message, please try back later.
Code: InvalidAuthenticationTokenTenant
Message: The access token is from the wrong issuer 'https://sts.windows.net/f8cdef31-a31e-4b4a-93e4-5f571e91255a/'. It must match the tenant 'https://sts.windows.net/a0a6d112-ab35-410a-b961-1280cf758bfc/' associated with this subscription. Please use the authority (URL) 'https://login.w

## 4.3 Configure online deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class.

In [ ]:
model = Model(path="../model-1/model/sklearn_regression_model.pkl")
env = Environment(
    conda_file="../model-1/environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model-1/onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

## 4.4 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# 5. Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [json](./model-1/sample-request.json) file. 

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="../model-1/sample-request.json",
)

# 6. Managing endpoints and deployments

## 6.1 Get details of the endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## 6.2 Get the logs for the new deployment
Get the logs for the green deployment and verify as needed

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

# 7. Delete the endpoint


In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)